## Impotando os Dados

In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
files.upload()

Saving Online Retail.xlsx to Online Retail.xlsx


In [ ]:
df = pd.read_excel("Online Retail.xlsx")

In [ ]:
df.head()

,CustomerID,StockCode,Description,Quantity
0,17850.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6
1,17850.0,71053,WHITE METAL LANTERN,6
2,17850.0,84406B,CREAM CUPID HEARTS COAT HANGER,8
3,17850.0,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6
4,17850.0,84029E,RED WOOLLY HOTTIE WHITE HEART.,6


In [ ]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [ ]:
df = df[["CustomerID","StockCode","Description","Quantity"]]

In [ ]:
df.to_csv('/content/drive/MyDrive/recomendacao/retail.csv')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/recomendacao/train_triplets.txt", sep='\t', lineterminator='\r')

In [ ]:
ratings = pd.read_csv('/content/drive/MyDrive/recomendacao/ratings_small.csv')

In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
user_ratings_df = ratings.pivot_table(index="userId",columns="movieId", values="rating")

In [ ]:
user_ratings_df.shape

(610, 9724)

## Medindo Esparsidade

In [ ]:
celulas_vazias = user_ratings_df.isnull().values.sum()
total = user_ratings_df.size
sparsity = celulas_vazias/total

In [ ]:
sparsity

0.9830003169443864

## SVD

In [ ]:
# Tomando a media de ratings de cada usuario
avg_ratings = user_ratings_df.mean(axis=1)

# Centralizando as ratings
user_ratings_centered = user_ratings_df.sub(avg_ratings, axis=0)

In [ ]:
user_ratings_centered.shape

(610, 9724)

In [ ]:
# Preenchendo com 0
user_ratings_centered.fillna(0, inplace=True)

In [ ]:
from scipy.sparse.linalg import svds
import numpy as np

In [ ]:
user_ratings_centered.shape

In [ ]:
# Decompoe
U, sigma, Vt = svds(user_ratings_centered)

In [ ]:
U.shape

(610, 6)

In [ ]:
Vt.shape

(6, 9724)

In [ ]:
# Converte sigma em uma matriz diagonal
sigma = np.diag(sigma)
print(sigma)

[[36.54895519  0.          0.          0.          0.          0.        ]
 [ 0.         37.95619249  0.          0.          0.          0.        ]
 [ 0.          0.         39.37050585  0.          0.          0.        ]
 [ 0.          0.          0.         41.77917206  0.          0.        ]
 [ 0.          0.          0.          0.         43.6224036   0.        ]
 [ 0.          0.          0.          0.          0.         76.20046537]]


## Recriando as Ratings

In [ ]:
# Produto de U com Sigma
U_sigma = np.dot(U, sigma)

In [ ]:
# Produto de (U com Sigma) com V transposto
U_sigma_Vt = np.dot(U_sigma, Vt)

# Print the result
print(U_sigma_Vt)

[[ 9.59343518e-02 -1.71945495e-02  2.27523739e-02 ... -1.34624568e-05
  -1.34624568e-05  5.89547842e-04]
 [-4.08968238e-04  4.54110162e-04 -4.93120168e-03 ... -8.36576329e-06
  -8.36576329e-06 -5.95822929e-06]
 [-2.95922628e-02  3.72046555e-03 -1.51491416e-02 ... -3.59273284e-05
  -3.59273284e-05 -1.52040897e-04]
 ...
 [ 3.12364266e-01 -2.95290977e-01 -4.52094620e-01 ... -3.59424467e-05
  -3.59424467e-05 -2.52639026e-05]
 [ 1.58856307e-02 -4.81933236e-03  1.37231199e-03 ...  1.98083300e-06
   1.98083300e-06  1.73775968e-04]
 [ 5.97439189e-01 -1.19549102e-01 -1.92823569e-01 ... -9.41640085e-04
  -9.41640085e-04  4.68139853e-03]]


In [ ]:
# Adiciona novamente as medias para recriarmos a matriz original
uncentered_ratings = U_sigma_Vt + avg_ratings.values.reshape(-1, 1)

In [ ]:
# Recria o dataframe
calc_pred_ratings_df = pd.DataFrame(uncentered_ratings, 
                                    index=user_ratings_df.index,
                                    columns=user_ratings_df.columns
                                   )

In [ ]:
calc_pred_ratings_df.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.462314,4.349185,4.389132,4.355828,4.304889,4.379954,4.339853,4.353081,4.371022,4.304191,...,4.366366,4.366333,4.366399,4.366399,4.366366,4.366399,4.366366,4.366366,4.366366,4.366969
2,3.947867,3.948730,3.943345,3.948975,3.954781,3.955014,3.950488,3.949590,3.946458,3.960781,...,3.948267,3.948247,3.948288,3.948288,3.948267,3.948288,3.948267,3.948267,3.948267,3.948270
3,2.406305,2.439618,2.420748,2.440964,2.468875,2.461135,2.451152,2.439374,2.424418,2.481790,...,2.435862,2.435774,2.435949,2.435949,2.435862,2.435949,2.435862,2.435862,2.435862,2.435745
4,3.616146,3.556886,3.576326,3.551370,3.500886,3.497739,3.521390,3.555289,3.562753,3.490580,...,3.555621,3.555780,3.555462,3.555462,3.555621,3.555462,3.555621,3.555621,3.555621,3.555394
5,3.653970,3.633074,3.646911,3.631662,3.620151,3.628583,3.628683,3.635113,3.640487,3.596743,...,3.636387,3.636443,3.636330,3.636330,3.636387,3.636330,3.636387,3.636387,3.636387,3.636556


In [ ]:
user_ratings_df.fillna(0)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Fazendo Recomendações com SVD

In [ ]:
# Recomendando pro user 5
user_4 = calc_pred_ratings_df.loc[4,:].sort_values(ascending=False)

In [ ]:
print(user_4)

## Usando SVD com Surprise

FunkSVD é uma variação do algoritmo SVD (Singular Value Decomposition) que é utilizado para lidar com valores ausentes em matrizes de dados. Ele foi proposto por Simon Funk durante o Netflix Prize, um desafio de recomendação de filmes promovido pela Netflix em 2006. A ideia principal do FunkSVD é preencher os valores ausentes na matriz de dados utilizando uma equação de aproximação baseada no SVD e no gradiente descendente. Isso permite que o algoritmo tenha uma melhor capacidade de lidar com dados esparsos, o que é comum em sistemas de recomendação.

In [ ]:
!pip install scikit-surprise -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [ ]:
# Carrega o dataframe
data = Dataset.load_builtin('ml-100k')

# Define o algoritmo
algo = SVD()

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [ ]:
# Treina e testa o algoritmotrain and test the algorithm
trainset, testset = train_test_split(data, test_size=.25)

In [ ]:
algo.fit(trainset)
predictions = algo.test(testset)

# evaluate the algorithm
accuracy.rmse(predictions)

RMSE: 0.9366


0.9366236236204732

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV

data = Dataset.load_builtin("ml-100k")

param_grid = {
    "n_epochs": [5, 10],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4, 0.6]
}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

0.9635054765364105
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


- n_epochs é o número de iterações de SGD, que é basicamente um método iterativo usado em estatística para minimizar uma função.

- lr_all é a taxa de aprendizado para todos os parâmetros, que é um parâmetro que decide quanto os parâmetros são ajustados em cada iteração.

- reg_all é o termo de regularização para todos os parâmetros, que é um termo de penalidade adicionado para evitar o overfitting.